In [1]:
import os
import datetime
import numpy as np
import pandas as pd
from functools import reduce

from PNL_tools.ProfitLossReporter import PNLReporter

## Data Preprocessing

In [2]:
##### read the files #####
os.chdir('C:/users/Ruibone/Desktop/fubon_stock/')

statement_dict = dict(all = pd.DataFrame())  # read yearly statement (2021 - 2024) into a dictionary
for index, year in enumerate(range(2021, 2025)):
    statement_dict[year] = pd.read_csv(f'statement_{year}.csv', header = 0, encoding = 'big5')
    statement_dict['all'] = pd.concat([statement_dict['all'], statement_dict[year]], axis = 0)
statement_dict['all'] = statement_dict['all'].reset_index(drop = True)


##### restructure the statement dataframe from dictionary #####
keep_column = ['成交日期', '\t交易類別', '\t股票名稱', '\t成交股數', '\t成交單價', '\t成交價金', '\t手續費', '\t交易稅', '\t淨收付金額']
revised_column = ['成交日期', '交易類別', '股票名稱', '成交股數', '成交單價', '成交價金', '手續費', '交易稅', '淨收付金額']
statement_df = statement_dict['all'].copy()[keep_column]
statement_df.columns = revised_column


##### add rows of ESPP (Employee Stock Purchase Plan) #####
statement_full = statement_df.copy()
statement_full.loc[len(statement_full)] = ['2022/12/08', '員工認股', '台積電(2330)', 37, 513.07, int(37*513.07), 0, 0, -int(37*513.07)]
statement_full.loc[len(statement_full)] = ['2023/03/08', '員工認股', '台積電(2330)', 65, 513.07, int(65*513.07), 0, 0, -int(65*513.07)]
statement_full.loc[len(statement_full)] = ['2023/06/08', '員工認股', '台積電(2330)', 69, 513.07, int(69*513.07), 0, 0, -int(69*513.07)]
statement_full.loc[len(statement_full)] = ['2023/09/08', '員工認股', '台積電(2330)', 66, 513.07, int(66*513.07), 0, 0, -int(66*513.07)]
statement_full.loc[len(statement_full)] = ['2023/11/08', '員工認股', '台積電(2330)', 47, 513.07, int(47*513.07), 0, 0, -int(47*513.07)]


##### correction for the date format & stock naming #####
statement_done = statement_full.copy()
statement_done['成交日期'] = pd.to_datetime(statement_full['成交日期'])
statement_done = statement_done.replace('復華S＆P500成長(00924)', '復華S&P500成長(00924)')
statement_done.tail(7)

,成交日期,交易類別,股票名稱,成交股數,成交單價,成交價金,手續費,交易稅,淨收付金額
191,2024-03-04,現股賣出,台積電(2330),100,710.00,71000,101.0,213.0,70686.0
192,2024-03-04,現股賣出,台積電(2330),50,711.00,35550,50.0,106.0,35394.0
193,2022-12-08,員工認股,台積電(2330),37,513.07,18983,0.0,0.0,-18983.0
194,2023-03-08,員工認股,台積電(2330),65,513.07,33349,0.0,0.0,-33349.0
195,2023-06-08,員工認股,台積電(2330),69,513.07,35401,0.0,0.0,-35401.0
196,2023-09-08,員工認股,台積電(2330),66,513.07,33862,0.0,0.0,-33862.0
197,2023-11-08,員工認股,台積電(2330),47,513.07,24114,0.0,0.0,-24114.0


## Profit Reporter

In [3]:
profit = PNLReporter(statement = statement_done)
profit.interval_selector()  # enddate = '2024-03-01'
profit.price_lookup()
profit.diviend_lookup()
profit.diviend_calculator()
profit.realized_calculator()
profit.unrealized_calculator()

profit.unrealized_summary['table']

StartDate: 2021-01-01; EndDate: 2024-03-04;
--------------------------除權息結果結算--------------------------
總配息收益:  15734
-----------------------------------------------------------------
--------------------------已實現損益結算--------------------------
投資成本:  501883
投資報酬:  552634
已實現總損益:  50751
已實現總損益率:  10.11%
-----------------------------------------------------------------
--------------------------未實現損益結算--------------------------
投資成本:  437938
投資現值:  502071
未實現總損益:  64133
未實現總損益率:  14.64%
-----------------------------------------------------------------


,持有股數,成交均價,付出成本,收盤價,未實現總價,未實現損益,未實現損益率
股票名稱,,,,,,,
元大台灣50正2(00631L),800,138.55,110843.0,176.35,141080.0,30237.0,27.28%
元大台灣高息低波(00713),2000,47.18,94359.0,51.9,103800.0,9441.0,10.01%
元大投資級公司債(00720B),200,36.70,7340.0,34.98,6996.0,-344.0,-4.69%
凱基優選高股息30(00915),3000,21.73,65189.0,23.35,70050.0,4861.0,7.46%
國泰20年美債正2(00688L),3000,9.69,29060.0,8.98,26940.0,-2120.0,-7.3%
大華優利高填息30(00918),1000,21.41,21410.0,22.41,22410.0,1000.0,4.67%
富邦NASDAQ正2(00670L),100,109.20,10920.0,114.85,11485.0,565.0,5.17%
富邦臺灣中小(00733),1000,50.30,50295.0,63.75,63750.0,13455.0,26.75%
群益半導體收益(00927),3000,16.17,48522.0,18.52,55560.0,7038.0,14.5%
